<a href="https://colab.research.google.com/github/shahaansshah/3253-083_Group9/blob/main/3253_Term_Project_v2%20(BK%20WIP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
pip install python-geohash

In [57]:
import json
import pandas as pd
import numpy as np
from geohash import encode
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler


# Assuming you have the JSON data in a file named 'data.json' in your Google Drive
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify the path to your JSON file
file_path = '/content/drive/MyDrive/Colab Notebooks/3253/ontario_housing_cleaned.json'

# Load the JSON data from the file line by line
records = []
with open(file_path, 'r') as f:
    for line in f:
        try:
            data = json.loads(line) # Load each line as a separate JSON object
            record = data.get('data') # Use .get() to handle potential missing 'data' key
            if record is not None: # Check if 'data' exists and is not None
                features = {}
                for feature_group in record.get('features', []): # Handle cases where 'features' might be missing
                    for feature_category in feature_group.get('value', []):
                        for feature in feature_category.get('value', []):
                            features[f"{feature_group['name']}_{feature_category['name']}_{feature['name']}"] = feature['value']

                record.update(features)  # Add the extracted features to the record
                record.pop('features', None)  # Remove the original nested features structure if it exists
                records.append(record)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line}") # Log any lines that fail to parse

# Create the DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
#df.to_csv('/content/drive/MyDrive/Colab Notebooks/3253/ontario_housing_cleaned.csv', index=False)


# Display the DataFrame
print(df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                         address subPremise  \
0  2480 Prince Michael Dr S #210       #210   
1                 1176 Grange Rd              
2                 3511 Post Road              
3            95 Dundas St W #513       #513   
4          3137 William Rose Way              

                                         fullAddress       division      city  \
0  2480 Prince Michael Dr S #210 Oakville, ON L6H...  Halton Region  Oakville   
1                1176 Grange Rd Oakville, ON L6H 1P6  Halton Region  Oakville   
2                3511 Post Road Oakville, ON L6H 7W5                 Oakville   
3           95 Dundas St W #513 Oakville, ON L6M 5N4  Halton Region  Oakville   
4         3137 William Rose Way Oakville, ON L6H 0T1  Halton Region  Oakville   

   cityCode closePrice closeDate daysOnMovoto  daysOnMovotoRaw  ... is3DTour  \
0    2506.0       Non

In [69]:
# Drop the specified columns
# These features are either irrelevant for predicting price, redundant, or contain mostly null values.
columns_to_drop = ['tnImgPath', 'imagesURL', 'photos', 'id', 'listingCoAgent', 'listingAgent',
                   'listingAgentLicense', 'listingOfficePhone',
                   'mlsDbNumber', 'mlsSysid', 'mls', 'mlsNumber', 'openHouses',
                   'officeColistName', 'officeListName', 'officeListPhone', 'photoCount', 'price', 'priceSeo', 'propertyTypeValue', 'propertyTypeDisplayName'
                   'state', 'status', 'pageUrlWithoutDomain', 'houseRealStatus', 'listingOfficeDescription', 'hoafee', 'hoafeeRaw', 'dppInactiveOnActive',
                   'dppInactive', 'priceChanged', 'priceChangedDate', 'updatedTime', 'hiddenByComplianceRule', 'dateHidden', 'propertyId', 'visibility',
                   'permitAvm', 'modificationTimestamp', 'createdAt', 'propertyDateHidden', 'imageDownloaderStatus', 'onMarketDateTime', 'priceChangeAmount', 'thumbnail',
                   'photoCount1', 'virtualTourLink', 'fsa', 'dppurl', 'listingByMovoto', 'labelDisplayName', 'listingPriceFormat', 'comparableHomes',
                   'listDateLLFormat', 'listDateLLFormat', 'listDateFormat', 'listDateUTC', 'pricePerSqft', 'pricePerSqftRaw', 'pricePerSqftIntRaw', 'isFavorite',
                   'petiteImagePath', 'propertyTypeNameUrl', 'closePrice', 'closeDate', 'daysOnMovoto', 'cityCode', 'listDate',
                   'state', 'soldDate', 'isHotHome', 'isSold', 'isPriceReduced', 'label', 'labelclass', 'Amenities  Utilities_Other_Pets Allowed (YN)',
                   'Amenities  Utilities_Utility_Utilities', 'Exterior_Building_# Total Stories', 'Exterior_Building_Building Amenities',
                   'Exterior_Building_Building Amenities', 'Exterior_Other_Exterior Features', 'Exterior_Other_Fencing', 'Exterior_Building_Foundation',
                   'Exterior_Parking_# Garage Spaces', 'Exterior_Parking_Drive', 'Exterior_Parking_Garage Features', 'Exterior_Parking_Has Basement (YN)',
                   'Exterior_Parking_Has Garage (YN)', 'Exterior_Parking_Parking Desc', 'Exterior_Parking_Parking Spot #',
                   'Interior_Bathrooms_# Full Baths', 'Interior_Bathrooms_# Half Baths', 'Interior_Bathrooms_# Three-Quarter', 'Interior_Bathrooms_# Total Bathrooms',
                   'Interior_Bedrooms_Family Room Available', 'Interior_Flooring_Flooring', 'Interior_Interior_Appliances', 'Interior_Interior_Has Fireplace (YN)',
                   'Interior_Interior_Laundry Information', 'Interior_Other_Interior Features', 'Location_Community_Community', 'Location_Community_Community Features',
                   'Location_Community_County', 'Location_Location features_Area', 'Location_Location features_Subdivision', 'Location_Location features_View',
                   'Location_Location features_Water Body Name', 'Location_Location features_Water Body Type', 'Location_Location features_Water Source',
                   'Location_Location features_Zoning Description', 'Location_Other_Directions', 'Location_Schools_Elementary School', 'Location_Schools_High School',
                   'Location_Schools_Middle School', 'Location_Schools_School District', 'Lot Land Details_Lot Information_Exposure',
                   'Lot Land Details_Lot Information_FarmAgriculture', 'Lot Land Details_Lot Information_Lot Desc', 'Lot Land Details_Lot Information_Lot Size Units',
                   'Lot Land Details_Lot Information_Water Features', 'Lot Land Details_Lot Information_Water Frontage', 'Overview_Lot_Approx Lot Size (Range)',
                   'Overview_Other_Approx Age', 'Overview_Other_Is Gated Community (YN)', 'Overview_Other_Is Horse Property (YN)', 'Overview_Other_New Construction (YN)',
                   'Overview_Other_Year Built', 'Overview_Other_HOA', 'Overview_Property_Approx Square Feet (Range)', 'Overview_Property_MLS #',
                   'Overview_Property_Status', 'Overview_Property_Storage Unit (Locker)', 'Overview_Property_Virtual Tour', 'Overview_Taxes_Tax Year', 'Overview_Taxes_Taxes',
                   'Rooms_Rooms Information_Movotorooms', 'SOA_HOUSEKEEPING_ATTRS_LISTING_SOURCE_URL_Listing Source URL',
                   'SOA_HOUSEKEEPING_ATTRS_LISTING_TYPE_Listing Type Identifier', 'virtualLink', 'is3DTour', 'isPriceUp', 'priceChange', 'priceChangeFriendlyPrice',
                   'lastListPriceRaw', 'lastListPrice', 'pricePerAcre', 'pricePerAcreRaw', 'pricePerAcreIntRaw', 'subPremise', 'fullAddress', 'division', 'daysOnMovotoRaw',
                   'description', 'lotSizeRaw', 'sqftTotalRaw', 'neighborhoodN', 'numBathroomsRaw', 'numBedroomsRaw', 'priceRaw', 'propertyTypeName','propertyTypeDisplayName',
                   'yearBuiltRaw', 'totalMonthlyFee', 'neighborhoodNGeoId', 'isVOWListing', 'addressRaw', 'address2', 'lotSizeUnit', 'sqftTotalUnit',
                   'Amenities  Utilities_Utility_Sewer Septic', 'Amenities  Utilities_Heating  Cooling_Heat Type', 'Amenities  Utilities_Other_Has Pool (YN)',
                   'Amenities  Utilities_Utility_Utility_Sewer Septic', 'Exterior_Building_Construction Materials', 'Exterior_Building_Roof',
                   'Exterior_Other_Other Structures', 'Exterior_Parking_# Parking Spaces', 'Interior_Bathrooms_# Three-Quarter Baths',
                   'Interior_Bedrooms_# of Above Grade Bedrooms', 'Interior_Bedrooms_# of Below Grade Bedrooms', 'Interior_Bedrooms_# of Rooms',
                   'Interior_More rooms_# of Kitchens', 'Lot Land Details_Lot Information_Lot Dimensions', 'Overview_Lot_Lot Size (Acres)', 'Overview_Other_Maintenance Fee',
                   'Overview_Property_Basement Information', 'Overview_Property_Building Size ', 'Overview_Property_Property Sub Type',
                   'city', 'neighborhoodName', 'priceChangeRaw', 'address', 'zipCode']
# Use errors='ignore' to avoid errors if a column doesn't exist, drop in-place
df.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/Colab Notebooks/3253/ontario_housing_cleaned2.csv', index=False)




In [59]:
# Display the DataFrame
#print(df.head(30).to_string())

In [98]:
df2 = df.copy()

#print(df2.head(30).to_string())


## Create Geohash Feature
#df2['geohash'] = df2.apply(lambda row: encode(row['latitude'], row['longitude'], precision=6), axis=1)
#df2.drop(columns=['latitude', 'longitude'], inplace=True, errors='ignore')

## Drop Rows with Missing 'listPrice', and show how many were dropped
print(f"Number of rows with missing 'listPrice': {df2['listPrice'].isna().sum()}")
df2.dropna(subset=['listPrice'], inplace=True)

# Drop outliers that will add noise to the model, not going to predict for outliers
# Drop propertyType in 'FARM', 'Land', 'Other', 'Multi Family'
df2 = df2[~df2['propertyType'].isin(['FARM', 'Land', 'Other', 'Multi_Family'])]
# Drop garage greater than 4
###df2 = df2[df2['garage'] <= 4]

# Fix column names
df2.columns = df2.columns.str.replace(r' +', '_', regex=True)

# Categorical variables levels cleanup
df2['Amenities_Utilities_Heating_Cooling_Cooling'] = df2['Amenities_Utilities_Heating_Cooling_Cooling'].str.replace(r'Ductless.*|.*Central .*', 'Y', regex=True)
df2['Amenities_Utilities_Heating_Cooling_Cooling'] = df2['Amenities_Utilities_Heating_Cooling_Cooling'].str.replace(r'^(?!.*Y).*', 'N', regex=True)
df2['Amenities_Utilities_Heating_Cooling_Heat_Source'] = df2['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'Electric.*', 'Electric', regex=True)
df2['Amenities_Utilities_Heating_Cooling_Heat_Source'] = df2['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'Natural gas.*', 'Gas', regex=True)
df2['Amenities_Utilities_Heating_Cooling_Heat_Source'] = df2['Amenities_Utilities_Heating_Cooling_Heat_Source'].str.replace(r'^(?!Electric|Gas).*', 'Other', regex=True)


## 1.6. One-Hot Encode Categorical Features
# One-hot encode categorical features: 'city', 'neighborhoodName', 'propertyType', 'parking', 'garage', 'Amenities  Utilities_Heating  Cooling_Cooling', 'Amenities  Utilities_Heating  Cooling_Heat Type'
categorical_features = ['propertyType', 'Amenities_Utilities_Heating_Cooling_Cooling', 'Amenities_Utilities_Heating_Cooling_Heat_Source']
df2 = pd.get_dummies(df2, columns=categorical_features, drop_first=True)*1

# Fix column names (because of values and their one-hot encoding)
df2.columns = df2.columns.str.replace(r' +', '_', regex=True)




## Impute Missing Values

# Impute missing year built with the median year built for the property type
#df2['yearBuilt'] = df2.groupby('propertyType')['yearBuilt'].transform(lambda x: x.fillna(x.median()))
# replace all non-numeric values in column df2[['yearBuilt']] with Nan
df2['yearBuilt'] = pd.to_numeric(df2['yearBuilt'], errors='coerce')
df2['numBedrooms'] = pd.to_numeric(df2['numBedrooms'], errors='coerce')
df2['numBathrooms'] = pd.to_numeric(df2['numBathrooms'], errors='coerce')
df2['lotSize'] = pd.to_numeric(df2['lotSize'], errors='coerce')
df2['sqftTotal'] = pd.to_numeric(df2['sqftTotal'], errors='coerce')

scaler = MinMaxScaler()
numerical_features = ['yearBuilt', 'latitude', 'longitude', 'numBedrooms', 'numBathrooms', 'lotSize', 'parking', 'sqftTotal', 'garage']
df2[numerical_features] = scaler.fit_transform(df2[numerical_features])

# Impute yearBuilt using KNNImputer
imputer = KNNImputer(n_neighbors=5)
# Scale yearBuilt for KNNImputer
df2['yearBuilt'] = imputer.fit_transform(df2[['yearBuilt', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'latitude', 'longitude']])

# Impute missing lot size based on median lot size for similar properties (same property type, city, and number of bedrooms)
#df2['lotSize'] = df2.groupby(['propertyType', 'numBedrooms', 'numBathrooms'])['lotSize'].transform(lambda x: x.fillna(x.median()))
# Impute lotSize based on numBedrooms and numBathrooms using KNNImputer

#imputer = KNNImputer(n_neighbors=5)
df2['lotSize'] = imputer.fit_transform(df2[['lotSize', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])

# Similar imputation strategy as 'lotSize'
#df2['sqftTotal'] = df2.groupby(['propertyType', 'numBedrooms', 'numBathrooms'])['sqftTotal'].transform(lambda x: x.fillna(x.median()))

#imputer = KNNImputer(n_neighbors=5)
df2['sqftTotal'] = imputer.fit_transform(df2[['sqftTotal', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])

#imputer = KNNImputer(n_neighbors=5)
df2['garage'] = imputer.fit_transform(df2[['garage', 'numBedrooms', 'numBathrooms', 'propertyType_Single_Family', 'propertyType_TOWNHOUSE', 'parking']])


# drop all rows that have NaN in any column
columns_to_check = ['garage', 'parking', 'numBedrooms', 'numBathrooms']
df2.dropna(subset=columns_to_check, inplace=True)

#print(df2[[]].isna().sum())
print(df2.head(30).to_string())
#print(df2.to_string())




Number of rows with missing 'listPrice': 0
    latitude  listPrice  longitude   lotSize  sqftTotal  numBathrooms  numBedrooms   parking  yearBuilt    garage  propertyType_Multi_Family  propertyType_Single_Family  propertyType_TOWNHOUSE  Amenities_Utilities_Heating_Cooling_Cooling_Y  Amenities_Utilities_Heating_Cooling_Heat_Source_Gas  Amenities_Utilities_Heating_Cooling_Heat_Source_Other
0   0.984358     995000   0.010590  0.007382   0.916550      0.028571     0.022222  0.045455   0.928000  0.026316                          0                           1                       0                                              1                                                    1                                                      0
1   0.983963    1190000   0.010911  0.007382   0.914343      0.028571     0.044444  0.068182   0.869333  0.026316                          0                           1                       0                                              1                      

In [99]:

# ## 1.7. Handle Outliers
# # Identify outliers in 'listPrice' based on IQR and remove them
# Q1 = df2['listPrice'].quantile(0.25)
# Q3 = df2['listPrice'].quantile(0.75)
# IQR = Q3 - Q1
# upper_bound = Q3 + 1.5 * IQR
# lower_bound = Q1 - 1.5 * IQR
# df2 = df2[(df2['listPrice'] >= lower_bound) & (df2['listPrice'] <= upper_bound)]

# 2. Feature Engineering

## 2.1. Combine Bathrooms and Bedrooms
# Create a new feature representing the total number of rooms (bedrooms + bathrooms)
#df2['totalRooms'] = df2['numBedrooms'] + df2['numBathrooms']

# 3. Split Data into Training and Testing Sets
X = df2.drop('listPrice', axis=1)
y = df2['listPrice']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Model Selection and Training
# Try different models: Linear Regression, Ridge, Lasso, ElasticNet, Decision Tree, Random Forest, Gradient Boosting
models = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'ElasticNet': ElasticNet(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

# 5. Hyperparameter Tuning
# Use GridSearchCV or RandomizedSearchCV to find the best hyperparameters for each model
param_grids = {
    'Ridge': {'alpha': np.logspace(-3, 3, 7)},
    'Lasso': {'alpha': np.logspace(-3, 3, 7)},
    'ElasticNet': {'alpha': np.logspace(-3, 3, 7), 'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
    'Decision Tree': {'max_depth': [None, 5, 10, 20], 'min_samples_split': [2, 5, 10]},
    'Random Forest': {'n_estimators': [100, 200, 500], 'max_depth': [None, 5, 10]},
    'Gradient Boosting': {'n_estimators': [100, 200, 500], 'learning_rate': [0.01, 0.1, 1]}
}

best_models = {}
for name, model in models.items():
    if name in param_grids:
        grid_search = GridSearchCV(model, param_grids[name], cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_
    else:
        model.fit(X_train, y_train)
        best_models[name] = model

# 6. Model Evaluation
# Evaluate the best models using metrics like MSE, R-squared, MAE, RMSE
for name, model in best_models.items():
    y_pred = model.predict(X_test)
    print(f'Model: {name}')
    print(f'MSE: {mean_squared_error(y_test, y_pred)}')
    print(f'R-squared: {r2_score(y_test, y_pred)}')
    print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
    print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}')
    print('---')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.556e+14, tolerance: 1.456e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.927e+14, tolerance: 1.397e+11
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.951e+14, tolerance: 1.426e

Model: Linear Regression
MSE: 453016274382.0671
R-squared: 0.4204703931238216
MAE: 354746.0576628946
RMSE: 673064.8366851942
---
Model: Ridge
MSE: 453164212655.0247
R-squared: 0.42028114029998864
MAE: 354284.44247689436
RMSE: 673174.7266906451
---
Model: Lasso
MSE: 452660225641.15564
R-squared: 0.4209258751859851
MAE: 354229.3434633549
RMSE: 672800.2865941391
---
Model: ElasticNet
MSE: 452660225641.15564
R-squared: 0.4209258751859851
MAE: 354229.3434633549
RMSE: 672800.2865941391
---
Model: Decision Tree
MSE: 272054630333.89877
R-squared: 0.651968986806692
MAE: 243943.7074293451
RMSE: 521588.56422845274
---
Model: Random Forest
MSE: 165728990423.98447
R-squared: 0.787988065551493
MAE: 204652.19100001908
RMSE: 407098.2564737713
---
Model: Gradient Boosting
MSE: 144060492686.33475
R-squared: 0.8157078996625887
MAE: 208379.70969156295
RMSE: 379553.01696381596
---
